In [1]:
import pandas as pd 

data_dir = '../data/'

animation_df = pd.read_csv(data_dir + 'kuka_20250302_183743.csv')
animation_df.head()

,Frame,Link,Parent,X_Position,X_Rotation,X_Velocity,Y_Position,Y_Rotation,Y_Velocity,Z_Position,Z_Rotation,Z_Velocity
0,0,LinkN0,Robot KUKA LBR Iiwa14,0.000000e+00,0.0,0.0,0.000000e+00,-0.0,0.0,0.000000,0.0,0.0
1,0,LinkN0-Axis,LinkN0,-3.489356e-08,0.0,0.0,0.000000e+00,-0.0,0.0,0.157500,0.0,0.0
2,0,LinkN1,LinkN0,-3.489356e-08,0.0,0.0,0.000000e+00,-0.0,0.0,0.157500,0.0,0.0
3,0,LinkN1-Axis,LinkN1,-1.553341e-09,0.0,0.0,4.268369e-08,-0.0,0.0,0.360067,0.0,0.0
4,0,LinkN2,LinkN1,-1.553341e-09,0.0,0.0,-7.719138e-05,0.0,0.0,0.360067,0.0,0.0


In [ ]:
print(f"frame df length: {len(animation_df):}")

frame df length: 6915


In [3]:
filtered_df = animation_df[~animation_df['Link'].str.contains('Axis')]
filtered_df.head()
print(f"filtered df length: {len(filtered_df):}")


filtered df length: 4149


In [4]:
links = ["LinkN0", "LinkN1", "LinkN2", "LinkN3", "LinkN4", "LinkN5", "LinkN6", "LinkN7"]
weird_data = filtered_df[~filtered_df['Link'].isin(links)]
print(f"link names: {weird_data['Link'].unique()}")

link names: ['LinkN6.002']


In [5]:
weird_link = "LinkN6.002"
print(f"number of rows containing {weird_link}: {len(filtered_df[filtered_df['Link'] == weird_link])}")

number of rows containing LinkN6.002: 461


In [6]:
cleaned_df = filtered_df[~filtered_df['Link'].str.contains(weird_link)]
print(len(cleaned_df))

3688


In [7]:
print(f"unique link names before cleaning: {animation_df['Link'].unique()}")
print(f"unique link names after cleaning: {cleaned_df['Link'].unique()}")


unique link names before cleaning: ['LinkN0' 'LinkN0-Axis' 'LinkN1' 'LinkN1-Axis' 'LinkN2' 'LinkN2-Axis'
 'LinkN3' 'LinkN3_Axis' 'LinkN4' 'LinkN4_Axis' 'LinkN5' 'LinkN5_Axis'
 'LinkN6' 'LinkN6.002' 'LinkN7']
unique link names after cleaning: ['LinkN0' 'LinkN1' 'LinkN2' 'LinkN3' 'LinkN4' 'LinkN5' 'LinkN6' 'LinkN7']


In [8]:
link_dfs = {}

for link in links:
    link_dfs[link] = cleaned_df[cleaned_df['Link'] == link].reset_index(drop=True)
    if link != "LinkN7":
        link_dfs[link]['qpos'] = link_dfs[link]['Y_Rotation']
        link_dfs[link]['qvel'] = link_dfs[link]['Y_Velocity']
    else:
        link_dfs[link]['qpos'] = link_dfs[link]['Z_Rotation'] 
        link_dfs[link]['qvel'] = link_dfs[link]['Z_Velocity']
        
    print(f"Created dataframe for {link} with {len(link_dfs[link])} rows")



Created dataframe for LinkN0 with 461 rows
Created dataframe for LinkN1 with 461 rows
Created dataframe for LinkN2 with 461 rows
Created dataframe for LinkN3 with 461 rows
Created dataframe for LinkN4 with 461 rows
Created dataframe for LinkN5 with 461 rows
Created dataframe for LinkN6 with 461 rows
Created dataframe for LinkN7 with 461 rows


In [9]:
link_dfs['LinkN6']

,Frame,Link,Parent,X_Position,X_Rotation,X_Velocity,Y_Position,Y_Rotation,Y_Velocity,Z_Position,Z_Rotation,Z_Velocity,qpos,qvel
0,0,LinkN6,LinkN5,-4.440887e-17,0.0,0.0,-0.06265,0.00000,0.0,1.18,0.0,0.0,0.00000,0.0
1,1,LinkN6,LinkN5,-4.440887e-17,0.0,0.0,-0.06265,0.00000,0.0,1.18,0.0,0.0,0.00000,0.0
2,2,LinkN6,LinkN5,-4.440887e-17,0.0,0.0,-0.06265,0.00000,0.0,1.18,0.0,0.0,0.00000,0.0
3,3,LinkN6,LinkN5,-4.440887e-17,0.0,0.0,-0.06265,0.00000,0.0,1.18,0.0,0.0,0.00000,0.0
4,4,LinkN6,LinkN5,-4.440887e-17,0.0,0.0,-0.06265,0.00000,0.0,1.18,0.0,0.0,0.00000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,456,LinkN6,LinkN5,-4.440887e-17,0.0,0.0,-0.06265,0.06418,0.0,1.18,0.0,0.0,0.06418,0.0
457,457,LinkN6,LinkN5,-4.440887e-17,0.0,0.0,-0.06265,0.06418,0.0,1.18,0.0,0.0,0.06418,0.0
458,458,LinkN6,LinkN5,-4.440887e-17,0.0,0.0,-0.06265,0.06418,0.0,1.18,0.0,0.0,0.06418,0.0
459,459,LinkN6,LinkN5,-4.440887e-17,0.0,0.0,-0.06265,0.06418,0.0,1.18,0.0,0.0,0.06418,0.0


In [10]:
# Start with the first link's frame, qpos and qvel
merged_df = link_dfs[links[0]][['Frame', 'qpos', 'qvel']].copy()
merged_df.rename(columns={
    'qpos': f'qpos_{links[0]}',
    'qvel': f'qvel_{links[0]}'
}, inplace=True)

# Merge with remaining links
for link in links[1:]:
    link_data = link_dfs[link][['Frame', 'qpos', 'qvel']].copy()
    link_data.rename(columns={
        'qpos': f'qpos_{link}',
        'qvel': f'qvel_{link}'
    }, inplace=True)
    merged_df = pd.merge(merged_df, link_data, on='Frame')

In [11]:
print(merged_df.head())

   Frame  qpos_LinkN0  qvel_LinkN0  qpos_LinkN1  qvel_LinkN1  qpos_LinkN2  \
0      0         -0.0          0.0         -0.0          0.0          0.0   
1      1         -0.0          0.0         -0.0          0.0          0.0   
2      2         -0.0          0.0         -0.0          0.0          0.0   
3      3         -0.0          0.0         -0.0          0.0          0.0   
4      4         -0.0          0.0         -0.0          0.0          0.0   

   qvel_LinkN2  qpos_LinkN3  qvel_LinkN3  qpos_LinkN4  qvel_LinkN4  \
0          0.0         -0.0          0.0          0.0          0.0   
1          0.0         -0.0          0.0          0.0          0.0   
2          0.0         -0.0          0.0          0.0          0.0   
3          0.0         -0.0          0.0          0.0          0.0   
4          0.0         -0.0          0.0          0.0          0.0   

   qpos_LinkN5  qvel_LinkN5  qpos_LinkN6  qvel_LinkN6  qpos_LinkN7  \
0         -0.0          0.0          0.0      

In [12]:
print(len(merged_df))

461


In [13]:
merged_df.to_csv(data_dir + 'kuka_formatted2.csv', index=False)